# Lesson 1: Simple ReAct Agent from Scratch

In [19]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [20]:
client = OpenAI(
    base_url="http://host.docker.internal:11434/v1",
    api_key="ollama"  # Ollama doesn't need a real API key
)

In [51]:
chat_completion = client.chat.completions.create(
    model="qwen2.5-coder:14b",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [52]:
chat_completion.choices[0].message.content

'Hello! How can I help you today?'

In [53]:
class Agent:
    def __init__(self, system = ""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completion = client.chat.completions.create(
            model = "qwen2.5-coder:14b",
            temperature = 0,
            messages = self.messages)
        return completion.choices[0].message.content

In [54]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [55]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [56]:
abot = Agent(prompt)

In [57]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the average weight of a Toy Poodle using the available action.

Action: average_dog_weight: Toy Poodle  
PAUSE


In [58]:
result = average_dog_weight("Toy Poodle")
print(result)

a toy poodles average weight is 7 lbs


In [59]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: a toy poodles average weight is 7 lbs'

In [60]:
result = abot(next_prompt)
print(result)

Answer: A Toy Poodle weighs approximately 7 lbs.


In [62]:
for m in abot.messages:
    print(m)


{'role': 'system', 'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'}
{'role': 'user', 'content': 'How much does a toy p

In [63]:
# Calculate current context usage
import json

def count_tokens_estimate(messages):
    """Rough estimate: ~4 chars = 1 token"""
    total_chars = sum(len(json.dumps(msg)) for msg in messages)
    return total_chars // 4

def check_context_window(messages, max_tokens=128000):
    """Check if we're approaching context limit"""
    current_tokens = count_tokens_estimate(messages)
    percentage = (current_tokens / max_tokens) * 100
    
    print(f"Current tokens (estimated): {current_tokens}")
    print(f"Max tokens: {max_tokens}")
    print(f"Usage: {percentage:.1f}%")
    print(f"Remaining: {max_tokens - current_tokens} tokens")
    
    if percentage > 90:
        print("⚠️ WARNING: Approaching context limit!")
    elif percentage > 70:
        print("⚠️ Consider summarizing older messages")
    else:
        print("✅ Context usage is healthy")
    
    return current_tokens


In [64]:
# Check current context for abot
check_context_window(abot.messages)

Current tokens (estimated): 338
Max tokens: 128000
Usage: 0.3%
Remaining: 127662 tokens
✅ Context usage is healthy


338

## More Accurate Token Counting (Optional)

For precise token counting, install `tiktoken`:
```bash
pip install tiktoken
```

Then use:
```python
import tiktoken

def count_tokens_accurate(messages, model="gpt-3.5-turbo"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens
```

**Common Context Windows:**
- `qwen2:7b` → ~8K tokens
- `llama3.2` → 128K tokens  
- GPT-3.5 → 16K tokens
- GPT-4 → 8K-128K tokens
- Claude 3 → 200K tokens

In [65]:
abot = Agent(prompt)

In [66]:
check_context_window(abot.messages)

Current tokens (estimated): 237
Max tokens: 128000
Usage: 0.2%
Remaining: 127763 tokens
✅ Context usage is healthy


237

In [67]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
    What is there combined weight"""
result = abot(question)
print(result)

Thought: To find the combined weight of the two dogs, I need to first determine the average weight of each breed using the `average_dog_weight` action, and then sum these weights.

Action: average_dog_weight: Border Collie
PAUSE


In [68]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [69]:
result = abot(next_prompt)
print(result)

Thought: Now that I have the average weight of the Border Collie, I need to find the average weight of the Scottish Terrier.

Action: average_dog_weight: Scottish Terrier
PAUSE


In [70]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [71]:
result = abot(next_prompt)
print(result)

Thought: I now have the average weights for both dogs. To find their combined weight, I will add these two values together.

Action: calculate: 37 + 20
PAUSE


In [72]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [73]:
result = abot(next_prompt)
print(result)

Answer: The combined weight of a Border Collie and a Scottish Terrier is approximately 57 lbs.


# Add loop

In [74]:
action_re = re.compile('^Action: (\w+): (.*)$') 

## Regex Pattern Explanation

`action_re = re.compile('^Action: (\w+): (.*)$')`

**Breaking it down:**
- `^` - Start of line
- `Action: ` - Literal text "Action: " (must match exactly)
- `(\w+)` - **Capture group 1**: One or more word characters (a-z, A-Z, 0-9, _)
  - This captures the action name like `calculate` or `average_dog_weight`
- `: ` - Literal colon and space
- `(.*)` - **Capture group 2**: Any characters (zero or more)
  - This captures the parameters like `4 * 7 / 3` or `Border Collie`
- `$` - End of line

**Example matches:**
- Input: `"Action: average_dog_weight: Border Collie"`
  - Group 1: `average_dog_weight`
  - Group 2: `Border Collie`
  
- Input: `"Action: calculate: 37 + 20"`
  - Group 1: `calculate`
  - Group 2: `37 + 20`

This regex parses the agent's "Action:" output to extract which function to call and with what arguments.

In [76]:
# Test the regex pattern
test_strings = [
    "Action: average_dog_weight: Border Collie",
    "Action: calculate: 37 + 20",
    "Thought: I should calculate the total",
    "Action: unknown_action: some params"
]

for test in test_strings:
    match = action_re.match(test)
    if match:
        action_name = match.group(1)
        params = match.group(2)
        print(f"✅ Matched: '{test}'")
        print(f"   Action: '{action_name}'")
        print(f"   Params: '{params}'")
    else:
        print(f"❌ No match: '{test}'")

✅ Matched: 'Action: average_dog_weight: Border Collie'
   Action: 'average_dog_weight'
   Params: 'Border Collie'
✅ Matched: 'Action: calculate: 37 + 20'
   Action: 'calculate'
   Params: '37 + 20'
❌ No match: 'Thought: I should calculate the total'
✅ Matched: 'Action: unknown_action: some params'
   Action: 'unknown_action'
   Params: 'some params'


In [84]:
def query(question, max_turns = 5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(f"Turn{i}: {result}\n")
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} [{}]".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation, "\n")
            next_prompt = "Observation: {}".format(observation)
        else:
            return
        

In [85]:
question = """I have 2 dogs, a border collie and a scottish terrier. 
What is their combined weight"""
query(question)

Turn1: Thought: To find the combined weight of the two dogs, I need to first determine the average weight of each breed using the `average_dog_weight` action, and then sum these weights.

Action: average_dog_weight: Border Collie
PAUSE

 -- running average_dog_weight [Border Collie]
Observation: a Border Collies average weight is 37 lbs 

Turn2: Thought: Now that I have the average weight of the Border Collie, I need to find the average weight of the Scottish Terrier.

Action: average_dog_weight: Scottish Terrier
PAUSE

 -- running average_dog_weight [Scottish Terrier]
Observation: Scottish Terriers average 20 lbs 

Turn3: Thought: With the weights of both dogs, I can now calculate their combined weight by adding the two values together.

Action: calculate: 37 + 20
PAUSE

 -- running calculate [37 + 20]
Observation: 57 

Turn4: Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.

